In [1]:
import pandas as pd

In [2]:
train=pd.read_csv('train.csv')

In [5]:
print("Training Set: "%train.columns,train.shape,len(train))

Training Set:  (31962, 3) 31962


In [6]:
test=pd.read_csv('test.csv')

In [12]:
print("Test Set:" %test.columns,test.shape,len(test))

Test Set: (17197, 2) 17197


In [13]:
## Data Cleaning

In [14]:
import re

In [15]:
def clean_text(df,text_field):
    df[text_field]=df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df
test_clean = clean_text(test, "tweet")
train_clean = clean_text(train, "tweet")


In [21]:
from sklearn.utils import resample

In [22]:
train_majority=train_clean[train_clean.label==0]
train_minority=train_clean[train_clean.label==1]


In [31]:
train_upsampled=resample(train_minority,replace=True,n_samples=len(train_majority),random_state=123)

In [32]:
train_upsampled=pd.concat([train_upsampled,train_majority])
train_upsampled['label'].value_counts()

1    29720
0    29720
Name: label, dtype: int64

In [33]:
##Creating a pipeline

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),])

In [36]:
from sklearn.model_selection import train_test_split


In [37]:
train_test_split?

In [38]:
X_train,X_test,Y_train,Y_test=train_test_split(train_upsampled['tweet'],train_upsampled['label'],random_state=0)

In [39]:
model=pipeline_sgd.fit(X_train,Y_train)
y_predict=model.predict(X_test)
from sklearn.metrics import f1_score
f1_score(Y_test,y_predict)

0.9695959461261503